# Projeto 3 - Pegando dados de DI da B3

### Selenium

* O Selenium simula uma pessoa em um navegador.
* Cliques, logins, digitações..

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time 
from datetime import datetime
from datetime import date

In [ ]:
#options = Options()
#options.headless = True

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) #driver da sua versão

In [ ]:
driver.get('https://www.google.com.br')

In [ ]:
data_di = date(2023, 10, 19)

In [ ]:
data_di = data_di.strftime("%d/%m/%Y")

url = f'''https://www2.bmf.com.br/pages/portal/bmfbovespa/boletim1/SistemaPregao1.asp?
pagetype=pop&caminho=Resumo%20Estat%EDstico%20-%20Sistema%20Preg%E3o&Data={data_di}
&Mercadoria=DI1'''

In [ ]:
driver.get(url)

### Pra achar elementos você deve inspecionar o site.

In [ ]:
local_tabela = '''/html/body/div/div[2]/form[1]/table[3]/tbody/tr[3]/td[3]/table'''
local_indice = '''/html/body/div/div[2]/form[1]/table[3]/tbody/tr[3]/td[1]/table'''        

In [ ]:
elemento = driver.find_element("xpath", local_tabela)
elemento_indice = driver.find_element("xpath", local_indice)

In [ ]:
html_tabela = elemento.get_attribute('outerHTML')
html_indice = elemento_indice.get_attribute('outerHTML')

In [ ]:
tabela = pd.read_html(html_tabela)[0]
indice = pd.read_html(html_indice)[0]

In [ ]:
driver.quit()

### Agora vamos tratar os dados..

In [ ]:
legenda = pd.Series(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                        index = ['F', 'G', 'H', 'J', 'K', 'M', 'N', 'Q', 'U', 'V', 'X', 'Z'])

In [ ]:
tabela

In [ ]:
tabela.columns = tabela.loc[0]

In [ ]:
tabela = tabela['ÚLT. PREÇO']
tabela = tabela.drop(0, axis = 0)
tabela

In [ ]:
indice.columns = indice.loc[0]
indice = indice.drop(0, axis = 0)

In [ ]:
tabela.index = indice['VENCTO']
tabela

In [ ]:
tabela = tabela.astype(int)
tabela = tabela[tabela != 0]
tabela = tabela/1000
tabela

In [33]:
lista_datas = []

for indice in tabela.index:

        letra = indice[0]
        ano = indice[1:3]

        mes = legenda[letra]

        data = f"{mes}-{ano}"

        data = datetime.strptime(data, "%b-%y")

        lista_datas.append(data)
        
tabela.index = lista_datas  

tabela = tabela/100
tabela

2023-11-01    0.12656
2023-12-01    0.12342
2024-01-01    0.12166
2024-02-01    0.12040
2024-03-01    0.11895
2024-04-01    0.11790
2024-05-01    0.11655
2024-06-01    0.11555
2024-07-01    0.11460
2024-08-01    0.11355
2024-09-01    0.11290
2024-10-01    0.11230
2025-01-01    0.11120
2025-04-01    0.11095
2025-07-01    0.11045
2025-10-01    0.11040
2026-01-01    0.11030
2026-04-01    0.11065
2026-07-01    0.11115
2026-10-01    0.11140
2027-01-01    0.11180
2027-04-01    0.11255
2027-07-01    0.11295
2027-10-01    0.11345
2028-01-01    0.11400
2028-04-01    0.11330
2028-07-01    0.11435
2028-10-01    0.11470
2029-01-01    0.11550
2030-01-01    0.11670
2031-01-01    0.11760
2032-01-01    0.11790
2033-01-01    0.11840
2036-01-01    0.11870
Name: ÚLT. PREÇO, dtype: float64